Install dependancies 

In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
torch.cuda.is_available()

True

## Imports

In [4]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback, IntervalStrategy
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from transformers import Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


## Set up Data Set

In [5]:
# Load and preprocess the training data
train_df = pd.read_csv('filtered_data_colab_clean.csv')

# Ensure that the text data is string
train_df['Description'] = train_df['Description'].astype(str)

train_texts = train_df['Description'].tolist()
train_labels = train_df['Policy_type'].tolist()

# Use the pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('mukund/privbert')

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# Define a PyTorch dataset
class TosdrDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
train_dataset = TosdrDataset(train_encodings, train_labels)


In [6]:
#train_df, val_df = torch.utils.data.random_split(train_dataset, [0.8, 0.2])


In [7]:
print("Length of Data Set", len(train_dataset))

print("Unique Classes (number of unique cases)", len(train_labels))


Length of Data Set 10015
Unique Classes (number of unique cases) 10015


In [8]:
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(train_dataset, [train_size,test_size])

## Model set up

In [9]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics_mod(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [10]:
# Initialize the pre-trained model, set num_labels to the number of unique cases
model = AutoModelForSequenceClassification.from_pretrained('mukund/privbert', num_labels=5)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    #auto_find_batch_size = True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    #logging_dir='./logs',
    logging_steps=50,

    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 5, # Evaluation and Save happens every 50 steps
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    learning_rate=2e-5,

    num_train_epochs=50,
    push_to_hub=False,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
    
)

# Define a trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset = val_set,
    compute_metrics = compute_metrics_mod,
)



Some weights of the model checkpoint at mukund/privbert were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mukund/privbert and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.

In [11]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

499900928

In [12]:
torch.cuda.reset_max_memory_allocated(device=None)

C:\Users\SNK5290\AppData\Roaming\Python\Python310\site-packages\torch\cuda\memory.py:303: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [ ]:
# Train the model
train_result = trainer.train()

# Load and preprocess the test data


C:\ProgramData\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8012
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 25050
  Number of trainable parameters = 124649477


Step,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 2003
  Batch size = 64
